**Pré-processamento - Perfis**
```
# Isto está formatado como código
```



In [ ]:
# Carregando as bibliotecas
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import silhouette_samples, silhouette_score, jaccard_score
import matplotlib.pyplot as plt

In [ ]:
#Carrega os datasets
arq_candidatos = '/data/D_ETL_IMO_EXTRACAO_SINE_ABERTO_TRABALHADORES_BA.csv'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
ds_perfis = pd.read_csv(arq_candidatos,encoding = "ISO-8859-1",sep=';')

In [14]:
ds_perfis.head()

,NACIONALIDADE,DEFICIENCIAS,BAIRRO,CEP,CODIGO_MUNICIPIO_IBGE,NOME_MUNICIPIO,UF,ESCOLARIDADE,ESTUDANTE,CURSOS_PROFISSIONALIZANTES,GRADUACOES,POS_GRADUACOES,IDIOMAS,HABILITACAO,VEICULOS,DISP_VIAJAR,DISP_DORMIR_EMP,DISP_AUSENTAR_DOMIC,PRETENSOES,MUNICIPIOS_INTERESSE
0,BRASILEIRA,NaN,ALTO DA CRUZ,42803113.0,290570,CAMACARI,BA,Médio Completo,N,NaN,NaN,NaN,NaN,Nenhum,N,N,N,N,"414140-Auxiliar de logistica(N,0,0,Indiferente...",NaN
1,BRASILEIRA,Um membro Inferior|Membros Superiores,LUIS EDUARDO MAGALHAES,47850000.0,291955,LUIS EDUARDO MAGALHAES,BA,Médio Completo,N,EMPILHADEIRA |INFORMATICA BASICA,NaN,NaN,NaN,B,N,N,N,N,511215-Cobrador de transportes coletivos (exce...,NaN
2,BRASILEIRA,NaN,NaN,40230700.0,292740,SALVADOR,BA,Médio Completo,N,NaN,NaN,NaN,NaN,Nenhum,N,N,N,N,"513615-Sushiman(N,105,0,Indiferente)",NaN
3,BRASILEIRA,NaN,CAMPO LIMPO,44034216.0,291080,FEIRA DE SANTANA,BA,Superior Incompleto,S,atendimento ao publico|verificar escolaridade ...,HISTORIA,NaN,NaN,Nenhum,N,N,N,N,252545-Analista financeiro (instituições finan...,NaN
4,BRASILEIRA,NaN,IGUAPE,45658335.0,291360,ILHEUS,BA,Médio Completo,N,NaN,NaN,NaN,NaN,Nenhum,N,N,N,N,514325-Oficial de serviços gerais na manutençã...,NaN


In [15]:
ds_perfis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 703323 entries, 0 to 703322
Data columns (total 20 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   NACIONALIDADE               703323 non-null  object 
 1   DEFICIENCIAS                5750 non-null    object 
 2   BAIRRO                      398465 non-null  object 
 3   CEP                         702928 non-null  float64
 4   CODIGO_MUNICIPIO_IBGE       703323 non-null  int64  
 5   NOME_MUNICIPIO              703323 non-null  object 
 6   UF                          703323 non-null  object 
 7   ESCOLARIDADE                703323 non-null  object 
 8   ESTUDANTE                   703323 non-null  object 
 9   CURSOS_PROFISSIONALIZANTES  157752 non-null  object 
 10  GRADUACOES                  63453 non-null   object 
 11  POS_GRADUACOES              1790 non-null    object 
 12  IDIOMAS                     27704 non-null   object 
 13  HABILITACAO   

In [16]:
#Visualiza domínio de municípios de interesse
ds_perfis['MUNICIPIOS_INTERESSE'].unique()

array([nan, '291840', '290570|290570|290570', ...,
       '293015|293015|293015|293015', '290780', '292205'], dtype=object)

In [17]:
#Visualiza coluna de PRETENSOES            
ds_perfis['PRETENSOES'].unique()      

array(['414140-Auxiliar de logistica(N,0,0,Indiferente)|521110-Operador de vendas (lojas)(N,0,0,Indiferente)|716610-Ajudante de pintor(N,0,6,Indiferente)|717020-Auxiliar de pedreiro(N,0,12,Indiferente)|784205-Auxiliar de linha de produção(N,0,0,Indiferente)',
       '511215-Cobrador de transportes coletivos (exceto trem)(N,5,0,Comercial)|517410-Porteiro(N,0,0,Indiferente)|517420-Vigia(N,0,0,Indiferente)|641010-Operador de máquinas de beneficiamento de produtos agrícolas(N,0,0,Indiferente)|715210-Pedreiro(N,0,0,Indiferente)|721105-Ferramenteiro(N,0,0,Indiferente)',
       '513615-Sushiman(N,105,0,Indiferente)', ...,
       '331110-Auxiliar de creche(N,0,12,Indiferente)|512105-Empregado  doméstico  nos serviços gerais(N,40,0,Indiferente)|512120-Empregado doméstico  diarista(N,0,120,Indiferente)|513315-Camareira de hotel(N,0,0,Indiferente)|514320-Auxiliar de limpeza(N,0,0,Indiferente)',
       '252305-Secretária(o) executiva(o)(N,66,0,Indiferente)|411005-Auxiliar administrativo(N,34,0,Ind

In [18]:
#Visualiza domínio de CURSOS_PROFISSIONALIZANTES
ds_perfis['CURSOS_PROFISSIONALIZANTES'].unique()

array([nan, 'EMPILHADEIRA |INFORMATICA BASICA',
       'atendimento ao publico|verificar escolaridade sem comprovação!!!!!!!!!!!!!!!',
       ..., 'camareira de hotel ',
       'pacote office 2010|telefonista/ RECEPCIONISTA- 2001',
       'auxiliar de dentista|tecnico em laboratorio analise e clinica'],
      dtype=object)

In [19]:
#Visualiza domínio de GRADUACOES
ds_perfis['GRADUACOES'].unique()

array([nan, 'HISTORIA', ' segurança do trabalho', ...,
       'administracao|engenharia civil', 'direitos',
       'biomedicina|especialização em hematologia e hemoterapia|pós graduação lato sensu em gestão de saúde'],
      dtype=object)

In [20]:
texto ='biomedicina|especialização em hematologia e hemoterapia|pós graduação lato sensu em gestão de saúde'
texto=texto.split("|")
for i in range(len(texto)):
  print("result:",texto[i][0:5])

result: biome
result: espec
result: pós g


In [21]:
#Função para calcular o número de itens
def get_qtd_itens(df):
    df_new=[]
    for i in range(len(df)):
      if(pd.isna(df[i])):
        df_new.append(0)
      else:
        texto=df[i].split("|") 
        df_new.append(len(texto)) 
    return df_new

In [22]:
def get_escolaridade_index(df_escolaridade):
    vet_escolaridade=['-1 - Não Identificado','0 - Nenhum','1 - Analfabeto','10 - Fundamental Incompleto', '5 - Fundamental Completo','6 - Médio Incompleto','7 - Médio Completo', '8 - Superior Incompleto', 'Superior Completo','11 - Mestrado', '12 - Doutorado']
    vet_escolaridade=['Não Identificado','Nenhum','Analfabeto','Fundamental Incompleto','Fundamental Completo','Médio Incompleto','Médio Completo','Superior Incompleto','Superior Completo','Especialização','Mestrado', 'Doutorado']
    df_esc=[]
    for i in range(len(df_escolaridade)):
      index=-1  
      for j in range(len(vet_escolaridade)):
        if(vet_escolaridade[j]==df_escolaridade[i]):
          index=j-1
      df_esc.append(index)
    return df_esc  

In [23]:
#Quantifica os cursos de graduação, profissionalizantes e total de idiomas
ds_perfis['N_GRADUACOES']=get_qtd_itens(ds_perfis['GRADUACOES'])
ds_perfis['N_CURSOS']=get_qtd_itens(ds_perfis['CURSOS_PROFISSIONALIZANTES'])
ds_perfis['N_IDIOMAS']=get_qtd_itens(ds_perfis['IDIOMAS'])

In [24]:
ds_perfis['N_ESCOLARIDADE']=get_escolaridade_index(ds_perfis['ESCOLARIDADE'])

In [25]:
#Preenche o campo MUNICIPIOS_INTERESSE nulo com o CODIGO_MUNICIPIO_IBGE do candidato
ds_perfis.MUNICIPIOS_INTERESSE.fillna(ds_perfis.CODIGO_MUNICIPIO_IBGE, inplace=True)

In [26]:
#Remove as colunas que não serão utilizadas exceto MUNICIPIOS_INTERESSE
ds_perfis=ds_perfis.drop(['NACIONALIDADE','DEFICIENCIAS', 'BAIRRO','CEP','NOME_MUNICIPIO','UF','ESTUDANTE','CURSOS_PROFISSIONALIZANTES','GRADUACOES','IDIOMAS','HABILITACAO','VEICULOS','DISP_VIAJAR','DISP_AUSENTAR_DOMIC','DISP_DORMIR_EMP','POS_GRADUACOES','ESCOLARIDADE','CODIGO_MUNICIPIO_IBGE'], axis=1)

In [27]:
ds_perfis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 703323 entries, 0 to 703322
Data columns (total 6 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   PRETENSOES            678769 non-null  object
 1   MUNICIPIOS_INTERESSE  703323 non-null  object
 2   N_GRADUACOES          703323 non-null  int64 
 3   N_CURSOS              703323 non-null  int64 
 4   N_IDIOMAS             703323 non-null  int64 
 5   N_ESCOLARIDADE        703323 non-null  int64 
dtypes: int64(4), object(2)
memory usage: 32.2+ MB


In [28]:
#Retira do dataset os perfis que não possuem pretenção de cargos
ds_perfis=ds_perfis[ds_perfis['PRETENSOES'].notna()]

In [29]:
#Resetando o índice 
ds_perfis.reset_index(drop=True,inplace=True)

In [30]:
# Trata o campo de texto de pretensões transformando-o de texto livre para vetor
df_new=[]
df=ds_perfis['PRETENSOES']
for i in range(len(df)):
    vet_pret=[]
    texto=df[i].split("|") 
    for j in range(len(texto)): #len(texto)
      texto[j]=texto[j][0:6]
      texto[j]=texto[j].replace('-', '')
      texto[j]=texto[j].replace('M', '')
      texto[j]=texto[j].replace('a', '')
      vet_pret.append(texto[j]) 
    df_new.append(vet_pret)  
ds_perfis['CBO_PRETENSAO']=df_new    

In [ ]:
# Remove a coluna de pretensões
ds_perfis=ds_perfis.drop(['PRETENSOES'], axis=1)

In [ ]:
ds_perfis.head()

In [ ]:
#Quebra os vetores de Pretenções duplicando as linhas de perfil do candidato
df_new=[]
for row in ds_perfis.itertuples(index=False):
  for j in range(len(row.CBO_PRETENSAO)):
    n_row=(row[0],row[1],row[2],row[3],row[4],row[5][j])
    df_new.append(list(n_row))


In [ ]:
ds_perfis = pd.DataFrame(df_new,columns=['MUNICIPIOS_INTERESSE','N_GRADUACOES','N_CURSOS','N_IDIOMAS','N_ESCOLARIDADE','CBO_PRETENSAO'])
ds_perfis = ds_perfis.astype({'MUNICIPIOS_INTERESSE': str, 'N_GRADUACOES': int, 'N_CURSOS': int, 'N_IDIOMAS': int, 'N_ESCOLARIDADE': int, 'CBO_PRETENSAO': str })


In [ ]:
ds_perfis.info()

In [ ]:
ds_perfis.head()

In [ ]:
# Trata o campo de texto de MUNICIPIOS_INTERESSE transformando-o de texto livre para vetor
df_new=[]
df=ds_perfis['MUNICIPIOS_INTERESSE']
for i in range(len(df)):
    vet_pret=[]
    texto=df[i].split("|") 
    for j in range(len(texto)):#len(texto)
      vet_pret.append(texto[j]) 
    df_new.append(vet_pret)  
ds_perfis['COD_IBGE_MUN_INTERESSE']=df_new    

In [ ]:
ds_perfis=ds_perfis.drop(['MUNICIPIOS_INTERESSE'], axis=1)

In [ ]:
ds_perfis.head()

In [ ]:
#Quebra os vetores de COD_IBGE_MUN_INTERESSE duplicando as linhas de perfil do candidato
df_new=[]
for row in ds_perfis.itertuples(index=False):
  for j in range(len(row.COD_IBGE_MUN_INTERESSE)):
    n_row=(row[5][j],row[0],row[1],row[2],row[3],row[4])
    df_new.append(list(n_row))

In [ ]:
ds_perfis.head()

In [ ]:
ds_perfis = pd.DataFrame(df_new,columns=['COD_IBGE_MUN_INTERESSE','N_GRADUACOES','N_CURSOS','N_IDIOMAS','N_ESCOLARIDADE','CBO_PRETENSAO'])

In [ ]:
ds_perfis = ds_perfis.astype({'COD_IBGE_MUN_INTERESSE': int, 'N_GRADUACOES': int, 'N_CURSOS': int, 'N_IDIOMAS': int, 'N_ESCOLARIDADE': int, 'CBO_PRETENSAO': str })

In [ ]:
ds_perfis.info()

In [ ]:
#Medidas descritivas
ds_perfis.describe()

In [ ]:
#Contagem de campos nulos
ds_perfis.isna().sum()

In [ ]:
#As 10 primeiras linhas
ds_perfis.head(10)

In [ ]:
# Considerando que o CBO está organizado em grupo e subgrupo de funções por área, então podemos propor um agrupamento por subgrupo até o nível 5
#ds_perfis['GRUPO_OCUPACAO']=np.trunc(ds_perfis['CBO_PRETENSAO']/10)
#ds_perfis['GRUPO_OCUPACAO']=ds_perfis.GRUPO_OCUPACAO.astype(np.int64)

In [ ]:
ds_perfis.head()
#Salva o data set para aplicação dos modelos de aprendizagem
ds_perfis.to_excel('perfis-ba.xlsx')